<a href="https://colab.research.google.com/github/paella55/03MAIR-Algoritmos-de-optimizacion/blob/master/Seminario/Seminario_(Grupos_Homogeneos)_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:  Jaime Díaz González <br>
Url: https://github.com/paella55/03MAIR-Algoritmos-de-optimizacion/blob/master/Seminario/Seminario_(Grupos_Homogeneos)_Algoritmos.ipynb<br>
Problema:
> 1. Elección de grupos de población homogéneos 

Descripción del problema: 
Se solicita diseñar un algoritmo para conseguir una agrupación de terneros en grupos de tratamientos que cumplan de la
mejor manera posible las especificaciones de la productora.

(*) La respuesta es obligatoria





                                        

 (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones? ****<br>



**¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.**




Respuesta:<br>
Para calcular el numero posible de soluciones es necesario algunas algo de combinatoria y saber contar.<br>
Suponemos que el número de ciudades es N y que partimos de una ciudad dada.
Podemos suponer un procedimiento que vaya construyendo todas las soluciones. Para el primer viaje disponemos de N-1 ciudades ya que debemos eliminar la ciudad de partida como posible candidata. Para la segunda ciudad a visitar disponemos de N-2 posiblidades. Por tanto ya tenemos (N-1)x(N-2) para visitar 2 ciudades. <br>
Si seguimos el razonamiento deducimos que hay (N-1)! (factoria de N-1) posibilidades.<br>
Puesto que el camino es circular(comienza y termina en la misma ciudad) debemos tener en cuenta que cada ruta tiene una ruta inversa semejante. La primera se convierte en la última, la segunda en la penultima y así sucesivamente. Por tanto sin no queremos tener en cuenta esta repeticion en total tenemos (N-1)!/2

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta:

Podemos etiquetar las ciudades con un indice único desde 1 a N para cada ciudad.

Es posible guardar las posibles soluciones en un vector de dimensión N. La primera coordenada será siempre la ciudad de partida.
La segunda coordenada será la segunda ciidad a visitar y así sucesivamente.

Si guardamos las ciudades en un diccionario de python podemos recuperar sus nombre a través de sus indices.

Debemos guardar las distancias o costes para ir desde cada par de ciudades. Aunque no nos lo han especificado en el problema, podemos suponer que el coste o distancia de ir de la ciudad i a j es el mismo que el de ir de j a i. Por tanto necesitamos una matriz simétrica para guardar estos valores.  De esta manera diremos que DISTANCIA[i][i] es la distancia de ir de i a j. En la diagonal evitamos disponemos 0(ceros)

### Creación de la estructura de datos

In [228]:
# librería externa para guardar los datos en tablas
import pandas as pd
from pandas import Series, DataFrame

import random
# liberia externa para manejo de vectores
import numpy as np

# Número de terneros
N = 80
# Número de tratamientos
tratamientos = 3
# Número de grupos
grupos = tratamientos*3

# Generamos datos
# Siguiendo la desviación estandar y la media del peso
pesos_terneros = np.random.uniform(low=35.0, high=39.1, size=(N,))
# Ahora hacemos machos y hembras
machos_hembras = []
for i in range(N):
  machos_hembras.append(random.choice('MH'))

def crear_tabla():
  # Me creo las columnas y le añado los valores
  terneros_dict = {"Peso": pesos_terneros, "Sexo": machos_hembras}
  # Creo la tabla que almacena los valores
  terneros_tabla = DataFrame(terneros_dict)
  return terneros_tabla

terneros_tabla = crear_tabla()
# Muestro 5 de entre todos
terneros_tabla.sample(5)

,Peso,Sexo
52,38.893676,H
32,36.058602,M
4,36.678406,H
14,35.234364,H
10,37.586123,M


### Filtrado de datos

In [0]:
# Ahora vamos a organizar los datos por sexo
terneros_machos = terneros_tabla[terneros_tabla['Sexo'] == 'M']
terneros_hembras = terneros_tabla[terneros_tabla['Sexo'] == 'H']

# Reiniciamos sus indices
terneros_machos = terneros_machos.reset_index(drop=True)
terneros_hembras = terneros_hembras.reset_index(drop=True)

# Ahora voy a ver cuantos terneros tengo que descartar, porque los grupos
# tienen que ser iguales en número.
machos_sobrantes = len(terneros_machos.index) % grupos
hembras_sobrantes = len(terneros_hembras.index) % grupos

# Descartamos los mayores y los menores pesos (outliers)
def descartar_terneros(df: DataFrame, num_descartar):
  
  for i in range(num_descartar):
    # En la iteración par, mayores
    if i % 2 == 0:
      indice = df['Peso'].idxmax()
      df = df.drop(df.index[indice])
      df = df.reset_index(drop=True)
    # En la iteración impar, menores
    else:
      indice = df['Peso'].idxmin()
      df = df.drop(df.index[indice])
      df = df.reset_index(drop=True)
  return df

# Descartamos terneros macho y reiniciamos los indices
terneros_machos = descartar_terneros(terneros_machos, machos_sobrantes)
terneros_machos = terneros_machos.reset_index(drop=True)
# Descartamos terneros hembra y reiniciamos los indices
terneros_hembras = descartar_terneros(terneros_hembras, hembras_sobrantes)
terneros_hembras = terneros_hembras.reset_index(drop=True)

### Resolución por fuerza bruta

In [230]:
# Intentamos encontrar los mínimos, para tener grupos iniciales mejores
def encontrar_minimos(df: DataFrame, num_minimos):
  # Creamos un nuevo DataFrame vacio
  df_minimos = pd.DataFrame()
  for i in range(num_minimos):
    # Buscamos el mínimo y se lo añadimos
    indice = df['Peso'].idxmin()
    df_minimos = df_minimos.append(df.iloc[indice])
    # Lo borramos para que no vuelva a aparecer y refrescamos los indices
    df = df.drop(df.index[indice])
    df = df.reset_index(drop=True)
  
  df_minimos = df_minimos.reset_index(drop=True)
  return df_minimos, df

tamanyo_grupos = len(terneros_machos.index)//tratamientos

# Creo una matriz tridimensional vacia para tener todos los tratamientos, todos los
# grupos y todos los individuos
matriz_grupos = np.zeros(shape=(tratamientos, grupos, tamanyo_grupos))

for t in range(tratamientos):
  for g in range(3):
    # Busco los mínimos y los añado al tratamiento y grupo
    (terneros_machos_min, terneros_machos) = encontrar_minimos(terneros_machos, tamanyo_grupos)

    print(terneros_machos)

    individuos = matriz_grupos[t][g]
    # Necesito los indices para poner los valores de los individuos
    indices = [i for i in range(individuos.size)]
    np.put(individuos, indices, terneros_machos_min['Peso'])
    # Finalmente pongo los individuos en la matriz
    matriz_grupos[t][g] = individuos

         Peso Sexo
0   36.718890    M
1   37.488814    M
2   37.765306    M
3   37.586123    M
4   37.330515    M
5   36.796091    M
6   37.420871    M
7   38.210421    M
8   36.917674    M
9   37.969117    M
10  38.320095    M
11  37.569465    M
12  37.669985    M
13  37.647477    M
14  38.082400    M
15  38.149074    M
16  38.308271    M
17  37.904071    M
        Peso Sexo
0  37.765306    M
1  38.210421    M
2  37.969117    M
3  38.320095    M
4  37.669985    M
5  38.082400    M
6  38.149074    M
7  38.308271    M
8  37.904071    M
Empty DataFrame
Columns: [Peso, Sexo]
Index: []


ValueError: ignored

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta:<br>
Dada una solución, por ejemplo posible_ruta=(1,2,3,4,5)<br>
Podemos diseñar una función que recorra la ruta y devuelva la suma de las distancias para la matriz de costes dada. Debemos tener en cuenta tambien el coste de ir desde la ultima ciudad a la primera para cerrar el circulo.<br>

Este es un problema de minimización


In [0]:
def calcular_distancia(ruta, COSTES):
  acumulado = 0                                #Valor acumulado de la ruta parcial
  for i in range(len(ruta)-1):                 #Para recorrer todas las ciudades del venctor
    acumulado += COSTES[ruta[i]][ruta[i+1]]    #Acumula
  return acumulado + COSTES[ruta[-1]][ruta[0]]  #Debemos tener en cuenta tambien el coste de ir desde la ultima ciudad a la primera para cerrar el circulo
    
print(posible_ruta)
calcular_distancia(posible_ruta, COSTES)

Diseña un algoritmo para resolver el problema por fuerza bruta

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

(*)Calcula la complejidad del algoritmo 

Respuesta

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [0]:
for i in range(5):
  COSTES[i][i] = 0
  for j in range(i+1, 5):
    COSTES[i][j] = random.randrange(200,500)
    COSTES[j][i] = COSTES[i][j]

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta